In [66]:
import os
import json

from pathlib import Path
from collections import defaultdict, Counter
from itertools import chain, starmap

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [117]:
source = pd.read_csv("../data/source.csv", parse_dates=["start", "end"])
source.head()

,Circuit,Weather Station,Wind Sustained,Gust Sustained,Thresholds (Sustained/ Gust),FPI Value,Reasons for De‐Energization,month,days,year,Wind Threshold,Gust Threshold,Wind Trigger Percentage,Gust Trigger Percentage,Combined Trigger Percentage,start day,end day,start,end
0,BOOTLEGGER,SCE Antelope Valley Fwy,16.0,41.0,31/46,13.98,"Exceeded Threshold, Exceeded FPI,",Sept,5-11,2020,31.0,46.0,51.612903,89.130435,89.130435,5,11,2020-09-05,2020-09-11
1,RED BOX,Chilao (CHOC1),16.0,46.0,31/46,14.01,"Exceeded Threshold, Exceeded FPI,",Sept,5-11,2020,31.0,46.0,51.612903,100.000000,100.000000,5,11,2020-09-05,2020-09-11
2,ENERGY,SCE Santa Susana Pass Rd,16.9,43.8,31/46,13.04,"High Wind Trend, Exceeded FPI,",Nov,29,2020,31.0,46.0,54.516129,95.217391,95.217391,29,29,2020-11-29,2020-11-29
3,SAND CANYON,SCE Magic Mtn Truck Trail,30.6,40.3,31/46,12.11,"High Wind Trend, Exceeded FPI,",Nov,29,2020,31.0,46.0,98.709677,87.608696,98.709677,29,29,2020-11-29,2020-11-29
4,DUKE,SCE Mount Davis,26.9,40.6,30/46,13.04,"High Wind Trend, Exceeded FPI,",Nov,29,2020,30.0,46.0,89.666667,88.260870,89.666667,29,29,2020-11-29,2020-11-29


In [171]:
ws = pd.read_csv("../data/weather/weather_stations.csv", parse_dates=True)
ws.head()

,StationNbr,Name,City,RegionalOffice,County,ConnectDate,DisconnectDate,IsActive,IsEtoStation,Elevation,GroundCover,HmsLatitude,HmsLongitude,ZipCodes,SitingDesc,Latitude,Longitude
0,1,Fresno/F.S.U. USDA,Fresno,South Central Region Office,Fresno,6/7/1982,9/25/1988,False,True,340,Grass,36º48'52N,-119º43'54W,"['93766', '93762', '93761', '93760', '93759', ...",NaN,36.814444,-119.73167
1,2,FivePoints,Five Points,South Central Region Office,Fresno,6/7/1982,12/31/2050,True,True,285,Grass,36º20'10N,-120º6'46W,['93624'],NaN,36.336222,-120.11291
2,3,Beach /Santa Cruz CO,Watsonville,South Central Region Office,Santa Cruz,5/30/1982,8/25/1986,False,True,10,Grass,36º52'50N,-121º47'36W,"['95077', '95076', '95075', '95019', '95018']",NaN,36.880556,-121.79333
3,4,Webb /Santa Cruz CO,Watsonville,South Central Region Office,Santa Cruz,5/30/1982,4/29/1988,False,True,230,Grass,36º58'21N,-121º43'34W,"['95077', '95076', '95075', '95019', '95018']",NaN,36.972500,-121.72611
4,5,Shafter,Shafter,South Central Region Office,Kern,6/1/1982,12/31/2050,True,True,360,Grass,35º31'57N,-119º16'54W,"['93263', '93280', '93388']",NaN,35.532556,-119.28179


In [235]:
wr = pd.read_csv("../data/weather/weather_report.csv", parse_dates=["Date", "Time"])

In [239]:
wr.Time.dtype

dtype('<M8[ns]')

In [163]:
wr.set_index(["Station", "Time"], inplace=True)
wr.head()

Date  Julian                       Hour  \
Station Time                                                                
2       2018-12-01 01:00:00 2018-12-01     335  0 days 01:00:00.000000000   
        2018-12-01 02:00:00 2018-12-01     335  0 days 02:00:00.000000000   
        2018-12-01 03:00:00 2018-12-01     335  0 days 03:00:00.000000000   
        2018-12-01 04:00:00 2018-12-01     335  0 days 04:00:00.000000000   
        2018-12-01 05:00:00 2018-12-01     335  0 days 05:00:00.000000000   

                            Standard ZipCodes   Scope  HlyAirTmp (F)  \
Station Time                                                           
2       2018-12-01 01:00:00  english    93624  hourly           41.6   
        2018-12-01 02:00:00  english    93624  hourly           43.9   
        2018-12-01 03:00:00  english    93624  hourly           41.7   
        2018-12-01 04:00:00  english    93624  hourly           39.2   
        2018-12-01 05:00:00  english    93624  hourly           37.0   

                             HlyDewPnt (F)  HlyEto (in)  HlyNetRad (Ly/day)  \
Station Time                                                                  
2       2018-12-01 01:00:00           39.8          0.0               -45.0   
        2018-12-01 02:00:00           39.5          0.0               -46.0   
        2018-12-01 03:00:00           39.1          0.0               -46.0   
        2018-12-01 04:00:00           37.9          0.0               -45.0   
        2018-12-01 05:00:00           36.2          0.0               -46.0   

                             HlyAsceEto (in)  HlyAsceEtr (in)  HlyPrecip (in)  \
Station Time                                                                    
2       2018-12-01 01:00:00              0.0              NaN             0.0   
        2018-12-01 02:00:00              0.0              NaN             0.0   
        2018-12-01 03:00:00              0.0              NaN             0.0   
        2018-12-01 04:00:00              0.0              NaN             0.0   
        2018-12-01 05:00:00              0.0              NaN             0.0   

                             HlyRelHum (%)  HlyResWind (MPH)  HlySoilTmp (F)  \
Station Time                                                                   
2       2018-12-01 01:00:00           93.0               2.6            57.4   
        2018-12-01 02:00:00           85.0               2.6            57.3   
        2018-12-01 03:00:00           90.0               1.2            57.1   
        2018-12-01 04:00:00           95.0               1.0            57.0   
        2018-12-01 05:00:00           97.0               2.0            56.8   

                             HlySolRad (Ly/day)  HlyVapPres (mBars)  \
Station Time                                                          
2       2018-12-01 01:00:00                 0.0                 8.3   
        2018-12-01 02:00:00                 0.0                 8.2   
        2018-12-01 03:00:00                 0.0                 8.1   
        2018-12-01 04:00:00                 0.0                 7.7   
        2018-12-01 05:00:00                 0.0                 7.2   

                             HlyWindDir (°)  HlyWindSpd (MPH)  
Station Time                                                   
2       2018-12-01 01:00:00           201.0               2.7  
        2018-12-01 02:00:00           239.0               3.4  
        2018-12-01 03:00:00             5.0               1.8  
        2018-12-01 04:00:00           127.0               1.7  
        2018-12-01 05:00:00           180.0               2.0

In [232]:
wr

Date  Julian                       Hour  \
Station Time                                                                
2       2018-12-01 01:00:00 2018-12-01     335  0 days 01:00:00.000000000   
        2018-12-01 02:00:00 2018-12-01     335  0 days 02:00:00.000000000   
        2018-12-01 03:00:00 2018-12-01     335  0 days 03:00:00.000000000   
        2018-12-01 04:00:00 2018-12-01     335  0 days 04:00:00.000000000   
        2018-12-01 05:00:00 2018-12-01     335  0 days 05:00:00.000000000   
...                                ...     ...                        ...   
264     2021-02-01 20:00:00 2021-02-01      32  0 days 20:00:00.000000000   
        2021-02-01 21:00:00 2021-02-01      32  0 days 21:00:00.000000000   
        2021-02-01 22:00:00 2021-02-01      32  0 days 22:00:00.000000000   
        2021-02-01 23:00:00 2021-02-01      32  0 days 23:00:00.000000000   
        2021-02-02 00:00:00 2021-02-01      32  1 days 00:00:00.000000000   

                            Standard                           ZipCodes  \
Station Time                                                              
2       2018-12-01 01:00:00  english                              93624   
        2018-12-01 02:00:00  english                              93624   
        2018-12-01 03:00:00  english                              93624   
        2018-12-01 04:00:00  english                              93624   
        2018-12-01 05:00:00  english                              93624   
...                              ...                                ...   
264     2021-02-01 20:00:00  english  96106, 96118, 96122, 96129, 96135   
        2021-02-01 21:00:00  english  96106, 96118, 96122, 96129, 96135   
        2021-02-01 22:00:00  english  96106, 96118, 96122, 96129, 96135   
        2021-02-01 23:00:00  english  96106, 96118, 96122, 96129, 96135   
        2021-02-02 00:00:00  english  96106, 96118, 96122, 96129, 96135   

                              Scope  HlyAirTmp (F)  HlyDewPnt (F)  \
Station Time                                                        
2       2018-12-01 01:00:00  hourly           41.6           39.8   
        2018-12-01 02:00:00  hourly           43.9           39.5   
        2018-12-01 03:00:00  hourly           41.7           39.1   
        2018-12-01 04:00:00  hourly           39.2           37.9   
        2018-12-01 05:00:00  hourly           37.0           36.2   
...                             ...            ...            ...   
264     2021-02-01 20:00:00  hourly           38.9           29.1   
        2021-02-01 21:00:00  hourly           40.3           28.2   
        2021-02-01 22:00:00  hourly           40.7           29.1   
        2021-02-01 23:00:00  hourly           40.1           30.7   
        2021-02-02 00:00:00  hourly           40.5           30.5   

                             HlyEto (in)  HlyNetRad (Ly/day)  HlyAsceEto (in)  \
Station Time                                                                    
2       2018-12-01 01:00:00          0.0               -45.0              0.0   
        2018-12-01 02:00:00          0.0               -46.0              0.0   
        2018-12-01 03:00:00          0.0               -46.0              0.0   
        2018-12-01 04:00:00          0.0               -45.0              0.0   
        2018-12-01 05:00:00          0.0               -46.0              0.0   
...                                  ...                 ...              ...   
264     2021-02-01 20:00:00          0.0               -39.0              0.0   
        2021-02-01 21:00:00          0.0               -40.0              0.0   
        2021-02-01 22:00:00          0.0               -39.0              0.0   
        2021-02-01 23:00:00          0.0               -39.0              0.0   
        2021-02-02 00:00:00          0.0               -39.0              0.0   

                             HlyAsceEtr (in)  HlyPrecip (in)  HlyRelHum (%)  \
Station Time             

In [198]:
wr[["HlyRelHum (%)", "HlyWindSpd (MPH)"]].describe(percentiles=np.arange(0,1,.01)).iloc[:15]

,HlyRelHum (%),HlyWindSpd (MPH)
count,525031.000000,526692.000000
mean,72.431256,3.823576
std,24.893570,7.934127
min,0.000000,0.300000
0%,0.000000,0.300000
1%,9.000000,1.000000
2%,14.000000,1.000000
3%,17.000000,1.000000
4%,20.000000,1.100000
5%,23.000000,1.100000


In [211]:
hum = wr["HlyRelHum (%)"]
(100 / hum[hum > 0]).describe(percentiles=np.arange(0,1,.01)).iloc[-10:]

91%      3.030303
92%      3.225806
93%      3.448276
94%      3.846154
95%      4.166667
96%      4.761905
97%      5.263158
98%      6.250000
99%      8.333333
max    100.000000
Name: HlyRelHum (%), dtype: float64

In [164]:
se_dates = source[["start", "end"]]
se_dates = se_dates[~se_dates.duplicated()].sort_values(by="start")
se_dates["label"] = se_dates.start.astype("str") + " - " + se_dates.end.astype("str")
se_dates.set_index("label", inplace=True)

In [166]:
psps_label = "2021-01-12 - 2021-01-21"
psps_date = se_dates.loc[psps_label]
wind_max = wr[(wr.Date >= psps_date.start) & (wr.Date <= psps_date.end)].groupby("Station")["HlyWindSpd (MPH)"].idxmax()
wind_max.dropna(inplace=True)

In [167]:
wind_max

Station
2        (2, 2021-01-19 01:00:00)
5        (5, 2021-01-19 13:00:00)
6        (6, 2021-01-18 14:00:00)
7        (7, 2021-01-19 06:00:00)
12      (12, 2021-01-19 14:00:00)
                  ...            
260    (260, 2021-01-12 11:00:00)
261    (261, 2021-01-12 23:00:00)
262    (262, 2021-01-18 13:00:00)
263    (263, 2021-01-13 00:00:00)
264    (264, 2021-01-19 13:00:00)
Name: HlyWindSpd (MPH), Length: 150, dtype: object

In [187]:
extreme_values = wr.loc[wind_max, ["HlyRelHum (%)", "HlyWindSpd (MPH)"]]
extreme_values["MaxTime"] = extreme_values.index.get_level_values(1)

In [190]:
ws.merge(extreme_values.droplevel(1), left_on="StationNbr", right_on="Station")

,StationNbr,Name,City,RegionalOffice,County,ConnectDate,DisconnectDate,IsActive,IsEtoStation,Elevation,GroundCover,HmsLatitude,HmsLongitude,ZipCodes,SitingDesc,Latitude,Longitude,HlyRelHum (%),HlyWindSpd (MPH),MaxTime
0,2,FivePoints,Five Points,South Central Region Office,Fresno,6/7/1982,12/31/2050,True,True,285,Grass,36º20'10N,-120º6'46W,['93624'],NaN,36.336222,-120.112910,28.0,18.1,2021-01-19 01:00:00
1,5,Shafter,Shafter,South Central Region Office,Kern,6/1/1982,12/31/2050,True,True,360,Grass,35º31'57N,-119º16'54W,"['93263', '93280', '93388']",NaN,35.532556,-119.281790,18.0,10.3,2021-01-19 13:00:00
2,6,Davis,Davis,North Central Region Office,Yolo,7/17/1982,12/31/2050,True,True,60,Grass,38º32'8N,-121º46'35W,"['95618', '95617', '95616']",NaN,38.535694,-121.776360,11.0,23.1,2021-01-18 14:00:00
3,7,Firebaugh/Telles,Firebaugh,South Central Region Office,Fresno,9/22/1982,12/31/2050,True,False,185,Grass,36º51'4N,-120º35'27W,"['93622', '93665', '93620']",NaN,36.851222,-120.590920,22.0,8.1,2021-01-19 06:00:00
4,12,Durham,Chico,Northern Region Office,Butte,10/19/1982,12/31/2050,True,True,130,Grass,39º36'31N,-121º49'28W,"['95974', '95958', '95938']",NaN,39.608639,-121.824430,18.0,11.3,2021-01-19 14:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,260,Montague,Montague,Northern Region Office,Siskiyou,9/20/2019,12/31/2050,True,True,2663,Grass,41º47'54N,-122º27'48W,"['96038', '96044', '96064', '96097']",NaN,41.798331,-122.463425,71.0,19.3,2021-01-12 11:00:00
146,261,Gazelle,Gazelle,Northern Region Office,Siskiyou,9/18/2019,12/31/2050,True,True,2743,Grass,41º32'2N,-122º31'56W,"['96034', '96038', '96064', '96094']",NaN,41.533989,-122.532279,76.0,31.8,2021-01-12 23:00:00
147,262,Linden,Linden,North Central Region Office,San Joaquin,2/26/2020,12/31/2050,True,True,111,Grass,38º3'56N,-121º4'18W,"['95240', '95236', '95227', '95237', '95252', ...",NaN,38.065692,-121.071747,16.0,15.9,2021-01-18 13:00:00
148,263,Smith River,Crescent City,Northern Region Office,Del Norte,10/7/2020,12/31/2050,True,True,22,Grass,41º53'41N,-124º9'54W,"['95532', '95567']",NaN,41.894592,-124.165043,99.0,19.2,2021-01-13 00:00:00


In [241]:
predictions = pd.read_csv("../data/predictions/predictions_24hr_jan19.csv")

In [242]:
predictions.head()

,Time,HlyAirTmp (F),HlyRelHum (%),HlyWindSpd (MPH),Station
0,2021-01-19 00:00:00,57.091297,44.099190,11.228603,2
1,2021-01-19 01:00:00,58.240654,54.797390,10.976135,2
2,2021-01-19 02:00:00,58.599210,63.684727,10.289221,2
3,2021-01-19 03:00:00,57.316475,70.779280,9.695071,2
4,2021-01-19 04:00:00,56.933968,76.562325,9.565497,2


In [244]:
crossings = pd.read_csv("../data/metrics/summaries/crossings_Jan19_24.csv")
crossings.rename(columns={"Prob of Crossing": "Crossing_Prob", "Actual Crossing Count": "Crossing_Count"}, inplace=True)

In [245]:
crossings.head()

,Station,Crossing_Prob,Crossing_Count
0,2,0.088,0.0
1,5,0.010,0.0
2,6,0.966,4.0
3,7,0.100,0.0
4,12,0.156,0.0
